In [2]:
# Torch
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import torchvision.utils
from torch.optim import lr_scheduler
from torch.autograd import Variable

# Torch network
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Other
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import os
import pandas as pd 
import boto3
import io
import cv2
from v_log import VLogger
import tqdm
from collections import OrderedDict
from importlib import reload  

# Imported files
import densenet

In [14]:
# PATHS

PATH_DATA = "/Users/david/TFM_DATA/spec"
data_files = os.listdir(PATH_DATA)

# Path Results
PATH_RESULTS_MODEL = "../model_results/"

# PATH triplets
PATH_TRIPLETS = os.path.join("..", "triplets", "triplets.csv")





# Name of experiment
EXP_NAME = "densenet_v2"
if EXP_NAME not in os.listdir(PATH_RESULTS_MODEL):
    #Create the folder of the EXP_NAME if not exists in the model_results folder
    os.mkdir(os.path.join(PATH_RESULTS_MODEL, EXP_NAME))
    
# Path LOGS
PATH_LOGS = os.path.join(PATH_RESULTS_MODEL, "logs")
if "logs" not in os.listdir(PATH_RESULTS_MODEL):
    #Create the folder of the EXP_NAME if not exists in the model_results folder
    os.mkdir(os.path.join(PATH_RESULTS_MODEL, "logs"))
PATH_LOG_FILE = os.path.join(PATH_LOGS, EXP_NAME + ".log")

In [15]:
# Start logging
log = VLogger(EXP_NAME, uri_log = PATH_LOG_FILE)

# Triplets: read csv

In [16]:
df = pd.read_csv(PATH_TRIPLETS, delimiter= ";")

# List
triplets_input = list(df["output"])
size_triplets_input = len(triplets_input)

# Sample
triplets_input_v1 = triplets_input[:1000]
size_triplets_input = len(triplets_input_v1)

# Classes

### Triplet Loaders

In [17]:
class TripletLoader(torch.utils.data.Dataset):
    
    # Constructor
    def __init__(self, triplets_input, transform):
        self.triplets = triplets_input
        self.transform = transform
        
    # Getter
    def __getitem__(self, index):
        
        # Given an index of the dataframe
        row = self.triplets[index]
        name_img1, name_img2, name_img3, _ = row.split(";")
        
        # Paths of .jpg according to the df row that we are processing
        path_img1 = os.path.join(PATH_DATA,name_img1)
        path_img2 = os.path.join(PATH_DATA,name_img2)
        path_img3 = os.path.join(PATH_DATA,name_img3)
                
        img1 = cv2.cvtColor(cv2.imread(path_img1), cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(cv2.imread(path_img2), cv2.COLOR_BGR2GRAY)
        img3 = cv2.cvtColor(cv2.imread(path_img3), cv2.COLOR_BGR2GRAY)
        
        # Resize to 256, 937
        img1 = cv2.resize(img1, (937, 256))
        img2 = cv2.resize(img2, (937, 256))
        img3 = cv2.resize(img3, (937, 256))
        
        # Transform (normalize)
        img1 = self.transform(img1)
        img2 = self.transform(img2)
        img3 = self.transform(img3)
        
        # Normalize from 0-255 to 0-1
        img1 /= 255.
        img2 /= 255.
        img3 /= 255.      
        
        return img1, img2, img3
    
    # overwrite the __len__ method
    def __len__(self):
        return len(self.triplets)
        
        return img1, img2, img3

In [18]:
def get_loader(triplets_input, cuda_bool, batch_size):
    
    # If CUDA enabled
    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_bool else {}
    
    # Data Loader
    train_data_loader = torch.utils.data.DataLoader(
        TripletLoader(triplets_input,transform = transforms.Compose([transforms.ToTensor()])),
        batch_size = batch_size, 
        shuffle = False, 
        **kwargs)
    
    return train_data_loader

### Model

In [19]:
def get_model(hyp, device, gpu_devices):
    """
    hyp: hyper parameters of the DenseNet in a dictionary
    Keys:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 3 or 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
            (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        small_inputs (bool) - set to True if images are 32x32. Otherwise assumes images are larger.
        efficient (bool) - set to True to use checkpointing. Much more memory efficient, but slower.
        
    gpu_devices: List of al the devices
    device: CPU or GPU
    """
    
    # Call the class for the DenseNet
    model = TripletNet(densenet.DenseNet, hyp)
    
    # Data Parallelization on GPU
    model = nn.DataParallel(model, device_ids = gpu_devices)
    
    # Put to device
    model = model.to(device)
    
    # Load weights if provided
    # (Disabled for this case, we will train from scratch)
    
    return model

### Train

In [20]:
class TripletNet(nn.Module):
    def __init__(self, DenseNetModel, hyperparamsDenseNet):
        super(TripletNet, self).__init__()
        self.embeddingNet = DenseNetModel(**hyperparamsDenseNet)

    def forward(self, i1, i2, i3):
        E1 = self.embeddingNet(i1)
        E2 = self.embeddingNet(i2)
        E3 = self.embeddingNet(i3)
        return E1, E2, E3

In [25]:
def train(train_data_loader, model, criterion, 
          optimizer, epoch, cuda_device):
    
    # Tell the object model to enter into the train mode
    model.train()
    
    # LOSS FUNCTION REINITIALIZE before next batch
    total_loss = 0
    
    for batch_idx, img_triplet in tqdm.tqdm(enumerate(train_data_loader)):
        
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        #               DATA LOAD
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        # Load the triplet (if batch = 10, then anchor_img size will be 10, 256, 937)
        anchor_img, pos_img, neg_img = img_triplet
        
        # Write the images into GPU ito cuda_device (if it's = 'cpu' then nothing occurs)
        anchor_img, pos_img, neg_img = anchor_img.to(cuda_device), pos_img.to(cuda_device), neg_img.to(cuda_device)
        
        # Convert images to Variables
        anchor_img, pos_img, neg_img = Variable(anchor_img), Variable(pos_img), Variable(neg_img)
        
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        #               MODEL FORWARD PROPAGATION
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        # Forward propagation
        E1, E2, E3 = model(anchor_img, pos_img, neg_img)
        
        # Distance of final embeddings
        dist_E1_E2 = F.pairwise_distance(E1, E2, 2) # same artist
        dist_E1_E3 = F.pairwise_distance(E1, E3, 2) # different artist
        
        
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        #               LOSS FUNCTION
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        # Create the target for the Margin Ranking Loss
        # in this case = -1 since the first input (dist_E1_E2) should have
        # a LOWER value than the second (dist_E1_E3)
        target = torch.FloatTensor(dist_E1_E2.size()).fill_(-1)
        target = target.to(cuda_device)
        target = Variable(target)
        
        # Call the loss function
        loss = criterion(dist_E1_E2, dist_E1_E3, target)
        total_loss += loss
        
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        #               OPTIMIZATION
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        #               LOG the loss
        ### ---------------------------------------- ###
        ### ---------------------------------------- ###
        # log the loss at each log_step batches (if 100, each 100 batches will monitor
        # the loss from 0-100, from 100-200, reseting the total_loss counter to 0
        log_step = 10 
        if (batch_idx % log_step == 0) and (batch_idx != 0):
            log.info('Train Epoch: {} [{}/{}] \t Loss: {:.4f}'.format(epoch, batch_idx, size_triplets_input,
                                                                      total_loss / log_step))
            total_loss = 0
        
    return anchor_img, pos_img, neg_img 

### Checkpoint

In [26]:
def save_model(epoch, model):
    model_to_save = {"epoch": epoch + 1,'state_dict': model.state_dict()}
    file_name = os.path.join(PATH_RESULTS_MODEL,EXP_NAME, "epoch_" + str(epoch) + ".pth")
    torch.save(model_to_save, file_name)
    log.info(f"Saved model checkpoint as: {file_name}")

# Main

In [27]:
#def main():
# Script: train.py

### --------------------------------------------- ###
### --------------------------------------------- ###
#               OVERALL PARAMETERS
### --------------------------------------------- ###
### --------------------------------------------- ###
epochs = 2
batch_size = 5
cuda_bool = False
checkpoint_freq_epoch = 1
triplets_input = triplets_input_v1 # LIST OF THE TRIPLETS separated by ;
learning_rate = 0.001

# KEY!!!! This is the KEY part, we won't set the difference of 1, since the triplets
# are constructed using a SIMILAR artist, then, we want to somehow preserve that similarity
# without affecting the real objective, which is to have similar embedding for the anchor and positive image
# compared to anchor and negative one, but, maybe anchor and negative one should be closer than anchor and a 
# "easy negative" one.
margin = 0.5 # margin for the Triplet Loss euclidean distance 

# Final EMBEDDING size
SIZE_EMBEDDING = 64

### --------------------------------------------- ###
### --------------------------------------------- ###

### --------------------------------------------- ###
### --------------------------------------------- ###
# CUDA
if cuda_bool:
    cuda_device = "cuda"
else:
    cuda_device = "cpu" #or "cuda"
    gpu_devices = None
### --------------------------------------------- ###


### --------------------------------------------- ###
### --------------------------------------------- ###
#  NETWORK: PARAMETERS of DENSENET
hyp = {
    "growth_rate":12, "block_config":(6,6,6), 
    "compression":0.5, "num_init_features":4, 
    "bn_size":4, # 4 times the growth rate
    "drop_rate":0, "num_classes":10, "small_inputs":True, "efficient":False,
    "input_channels": 1, "SIZE_EMBEDDING": SIZE_EMBEDDING
}
### --------------------------------------------- ###


### --------------------------------------------- ###
### --------------------------------------------- ###
# MODEL: Build model

model = get_model(hyp, cuda_device, gpu_devices)
### --------------------------------------------- ###


### --------------------------------------------- ###
### --------------------------------------------- ###
# CRITERION and OPTIMIZER: Retrieve the list of params in that moment of the network optimizer
params = []
for key, value in dict(model.named_parameters()).items():
    if value.requires_grad:
        params += [{'params': [value]}]


criterion = torch.nn.MarginRankingLoss(margin=margin)
optimizer = optim.Adam(params, lr=learning_rate)


### --------------------------------------------- ###
### --------------------------------------------- ###
#          EPOCH TRAINING LOOP
### --------------------------------------------- ###
### --------------------------------------------- ###

for epoch in range(1, epochs + 1):
    
        
    # Log
    log.info("---------------------------------------------")
    log.info("---------------------------------------------")
    log.info(f"----------   START EPOCH {epoch}  ----------")
    log.info("---------------------------------------------")
    log.info("---------------------------------------------")

    # Get the data loader object
    train_data_loader = get_loader(triplets_input, cuda_bool, batch_size)

    # Train the model at each batches
    anchor_img, pos_img, neg_img  = train(train_data_loader,
                                          model, criterion, 
                                          optimizer, epochs, 
                                          cuda_device) #TODO

    # Save model at each epoch
    save_model(epoch, model)
    
    # Log
    log.info("---------------------------------------------")
    log.info("---------------------------------------------")
    log.info(f"----------     END EPOCH {epoch}  ----------")
    log.info("---------------------------------------------")
    log.info("---------------------------------------------")


INFO     [2020-08-10 19:51:57] densenet_v2          | ---------------------------------------------
INFO     [2020-08-10 19:51:57] densenet_v2          | ---------------------------------------------
INFO     [2020-08-10 19:51:57] densenet_v2          | ----------   START EPOCH 1  ----------
INFO     [2020-08-10 19:51:57] densenet_v2          | ---------------------------------------------
INFO     [2020-08-10 19:51:57] densenet_v2          | ---------------------------------------------
4it [08:17, 124.30s/it]


KeyboardInterrupt: 